In [ ]:
"""
@author: ChewingGumKing_OJF
"""
import json
import os
import sys
import time
from dataclasses import dataclass
from datetime import datetime
#loads necessary libraries
from typing import Union, Any, Tuple, Sequence, List
import re
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import warnings
import requests
from GlobalVariable import *
from GlobalFunctions import GlobalFunctions

error: str = ''
options = webdriver.ChromeOptions()

#options.add_argument("--headless")

path = GlobalVariable.ChromeDriverPath
driver = webdriver.Chrome(options=options, executable_path=path)

#*******************************************************************************************************************
import logging
def creating_log(script_name: str):

    log_folder_path = 'log_folder'

    if os.path.exists(log_folder_path):
        for files in os.listdir(log_folder_path):
            if files == 'jupyter_run.log':
                os.remove(os.path.join(os.getcwd(), log_folder_path, files))
    else:
        os.makedirs(log_folder_path)

    log_path = os.path.join(os.getcwd(), log_folder_path, f'{script_name}.log')

    logger = logging.getLogger(script_name)
    logger.setLevel(logging.DEBUG)
    log_handler = logging.FileHandler(log_path)
    log_format = logging.Formatter(
        '%(asctime)s -- %(name)s -- %(levelname)s -- %(message)s \n\n')
    log_handler.setFormatter(log_format)
    logger.addHandler(log_handler)
    logger.info('Log reporting is instantiated.')

    return logger


logger = creating_log('jupyter_run')
#*******************************************************************************************************************

In [ ]:
class ScrapeEvent:
    """ the codebase design uses a Class with it Methods as function scraping singular data(some more,
        in the case of going inside the page just once). It returns the data to a it caller which is handled by a context manager.
    """

    browser: WebDriver = driver
    wait_5sec: WebDriverWait = WebDriverWait(browser, 5)
    error_msg_from_class: str = ''

    def __enter__(self):
        "Handles the contex manager."
        return self

    def __exit__(self, exc_type=None, exc_value=None, exc_tb=None):
        self.browser.quit()

    def get_link(self, link: str) -> None:
        "Hanles the teardown of the context manager."
        self.browser.get(link)

    def dispatch(self, class_name:str, finder:webdriver = By.CSS_SELECTOR):
        "API calls for Selenium webdriver.find_element"
        return self.browser.find_element(finder, class_name)

    def dispatchList(self, class_name:str, finder: webdriver = By.CSS_SELECTOR):
        "API calls for Selenium webdriver.find_elements"
        return self.browser.find_elements(finder, class_name)


    def get_done(self) -> Any:
            "Returns a list of all urls"
            #self.browser.get(url)
            time.sleep(3)
            try:
                
                self.browser.find_element(By.CSS_SELECTOR, '#onetrust-accept-btn-handler').click() 
            except StaleElementReferenceException: pass
            except NoSuchElementException: pass
            except: pass

            flag = True
            while flag:
                try:
                    self.browser.find_element(By.CSS_SELECTOR, '.btn__more').click()
                    flag = True 
                    time.sleep(2)
                except NoSuchElementException or StaleElementReferenceException:
                    flag = False

            container:list = []
            all_event = self.browser.find_elements(By.CSS_SELECTOR, '.event.lg-3.md-6.xs-12')
            for i in range(len(all_event)):
                i += 1
                event_date = self.browser.find_elements(By.CSS_SELECTOR, f'.event.lg-3.md-6.xs-12:nth-child({i}) .date time')
                
                try:
                    each_event_dates = [each.get_attribute('datetime') for each in event_date]
                    print('one\n\n\n', each_event_dates )
                except:
                    each_event_dates = [each.get_dom_attribute('datetime') for each in event_date]          

                last_date = each_event_dates[1].split('T')[0]
                rf_datetime = datetime.strptime(last_date, '%Y-%m-%d')
                if rf_datetime > datetime.now():

                    url = self.browser.find_element(By.CSS_SELECTOR, f'.event.lg-3.md-6.xs-12:nth-child({i}) a').get_attribute('href')

                    event_name = self.browser.find_element(By.CSS_SELECTOR, f'.event.lg-3.md-6.xs-12:nth-child({i}) .title').text
                    
                    event_venue = self.browser.find_element(By.CSS_SELECTOR, f'.event.lg-3.md-6.xs-12:nth-child({i}) .venue').text

                    event_type = self.browser.find_element(By.CSS_SELECTOR, f'.event.lg-3.md-6.xs-12:nth-child({i}) .feature.topic').text.replace('\n', '').strip()

                    dates = each_event_dates[0].split('T')[0], last_date

                    container.append((url, event_name, event_venue, event_type , dates))
                    
                else: pass

            return container
            





In [ ]:
url = 'https://www.imperial.ac.uk/whats-on/'

In [ ]:
with ScrapeEvent() as handler:
    handler.browser.implicitly_wait(5)

    try:
        links = handler.get_link(url)
    except Exception as e:
        logger.error('Connection Error', exc_info=True)
        error += str(e) 
        
    try:
        print('-----********************-----')
        res = handler.get_done()
        print(res)
    except Exception as e:
        logger.error('Function failed', exc_info=True)
        error +=  str(e) 

    error += handler.error_msg_from_class


print('-----********************-----')
print('DONE')


In [1]:
import re
venue = 'Hotel5060'
if re.search('Hotel\d+',venue):
    venue=re.search('.+Hotel',venue).group()
    print(venue)

AttributeError: 'NoneType' object has no attribute 'group'